## Lies `cars.csv` in einen DataFrame

In [10]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/cars.csv")
df.head()

,price,yearOfRegistration,powerPS,kilometer,model,fuelType,name
0,1450,1997,75,90000,andere,benzin,Toyota_Toyota_Starlet_1._Hand__TÜV_neu
1,13100,2005,280,5000,golf,benzin,R32_tauschen_oder_kaufen
2,4500,2008,87,90000,yaris,benzin,Toyota_Yaris_1.3_VVT_i
3,6000,2009,177,125000,3er,diesel,320_Alpinweiss_Kohlenstoff
4,3990,1999,118,90000,3er,benzin,BMW_318i_E46_+++_1._Hand_+++_Liebhaberfahrzeug


## Data Cleaning

* entferne die Features `model` und `name`
* entferne Observations mit `NaN`-Einträgen
* entferne Observations, deren `fuelType` nicht `benzin` oder `diesel` ist
* Führe ein One-Hot-Encoding für `fuelType` durch

In [11]:
df = df.drop(columns=["model", "name"])


In [12]:
df = df.dropna()
df.isna().sum()

price                 0
yearOfRegistration    0
powerPS               0
kilometer             0
fuelType              0
dtype: int64

In [13]:
df = df.query("fuelType == 'benzin' or fuelType == 'diesel'")
df['isDiesel'] = df['fuelType'].replace({'benzin':0, 'diesel':1})
df.drop('fuelType')

In [14]:
df.head()

,price,yearOfRegistration,powerPS,kilometer,fuelType,isDiesel
0,1450,1997,75,90000,benzin,0
1,13100,2005,280,5000,benzin,0
2,4500,2008,87,90000,benzin,0
3,6000,2009,177,125000,diesel,1
4,3990,1999,118,90000,benzin,0


## Training

* Verwende 20% Testdaten und `random_state=42`
* predicte `price`
* Verwende eine LinearRegression und ermittle, welches Feature einen negativen Einfluss auf den Preis hat
* Verwende einen RandomForestRegressor und ermittle das wichtigste Feature

In [15]:
X = df.drop(columns='price')
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
model = LinearRegression()
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'diesel'

In [ ]:
df.corr()['price'][1:   ]

## Evaluierung

* Ermittle den mean squared error für beide Modelle
* Performt eines der models besser, wenn die Daten skaliert werden?

### Tuning

Finde ein model, welches am Testset einen mse < 3200 aufweist